<a href="https://colab.research.google.com/github/prudhvibhukaya/Survey-Feedback-Response-Bot/blob/main/Sentiment_Analysis_for_Proactive_Customer_Engagement.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
from textblob import TextBlob
import pandas as pd

In [10]:
# Replace 'your_excel_file.xlsx' with the name of your uploaded Excel file
excel_file_path = '/content/Survey sample.xlsx'

try:
    df_excel = pd.read_excel(excel_file_path)
    display(df_excel.head())
except FileNotFoundError:
    print(f"Error: The file '{excel_file_path}' was not found. Please make sure you have uploaded the file and the path is correct.")
except Exception as e:
    print(f"An error occurred while reading the Excel file: {e}")

,Timestamp,Customer Type,Rating (1–5),Feedback
0,2025‑06‑18,Enterprise Client,4.0,"“Intuitive UI, great onboarding, but initial i..."
1,2025‑03‑12,Mid‑Market,5.0,“Excellent onboarding process and super helpfu...
2,2024‑12‑14,Large Enterprise,4.5,"“Highly customizable platform, but missing fea..."
3,2024‑02‑14,Large Enterprise,5.0,“Great solution for CSRD & LkSG with strong su...
4,2024‑06‑12,Enterprise,3.5,"“Some performance slowdowns, still evolving fe..."


In [11]:
# Apply sentiment analysis to the 'Feedback' column in df_excel
df_excel['sentiment_score'] = df_excel['Feedback'].apply(lambda x: TextBlob(str(x)).sentiment.polarity)
df_excel['sentiment_label'] = df_excel['sentiment_score'].apply(lambda x: 'positive' if x >= 0.1 else ('negative' if x <= -0.1 else 'neutral'))

# Sort the DataFrame by 'sentiment_score' in descending order
df_sorted = df_excel.sort_values(by='sentiment_score', ascending=False)

# Display the sorted DataFrame
display(df_sorted[['Feedback', 'sentiment_score', 'sentiment_label', 'Customer Type', 'Timestamp']])

,Feedback,sentiment_score,sentiment_label,Customer Type,Timestamp
3,“Great solution for CSRD & LkSG with strong su...,0.616667,positive,Large Enterprise,2024‑02‑14
1,“Excellent onboarding process and super helpfu...,0.511111,positive,Mid‑Market,2025‑03‑12
4,"“Some performance slowdowns, still evolving fe...",0.500000,positive,Enterprise,2024‑06‑12
0,"“Intuitive UI, great onboarding, but initial i...",0.400000,positive,Enterprise Client,2025‑06‑18
2,"“Highly customizable platform, but missing fea...",-0.020000,neutral,Large Enterprise,2024‑12‑14


# Task
Create an automation process that monitors customer feedback from the file "customer_feedback.xlsx", calculates sentiment scores, identifies feedback with a sentiment score less than or equal to 0.4, and automatically sends an email to the CS team with the details of these feedback entries.

## Filter data

### Subtask:
Filter the `df_excel` DataFrame to select feedback entries where the `sentiment_score` is less than or equal to 0.4.


**Reasoning**:
Filter the DataFrame based on the sentiment score and display the result.



In [12]:
negative_feedback_df = df_excel[df_excel['sentiment_score'] <= 0.4]
display(negative_feedback_df.head())

,Timestamp,Customer Type,Rating (1–5),Feedback,sentiment_score,sentiment_label
0,2025‑06‑18,Enterprise Client,4.0,"“Intuitive UI, great onboarding, but initial i...",0.40,positive
2,2024‑12‑14,Large Enterprise,4.5,"“Highly customizable platform, but missing fea...",-0.02,neutral


## Prepare email content

### Subtask:
Format the relevant information (e.g., Feedback, Customer Type, Timestamp, Sentiment Score) from the filtered data into a suitable structure for an email.


**Reasoning**:
Select the relevant columns and format the data as a string for the email body.



In [13]:
email_body_content = negative_feedback_df[['Feedback', 'Customer Type', 'Timestamp', 'sentiment_score']].to_string(index=False)
print(email_body_content)

                                                                         Feedback     Customer Type  Timestamp  sentiment_score
    “Intuitive UI, great onboarding, but initial integration was time‑intensive.” Enterprise Client 2025‑06‑18             0.40
“Highly customizable platform, but missing feature for automated BAFA reporting.”  Large Enterprise 2024‑12‑14            -0.02


## Implement email sending

### Subtask:
Write code to send emails to the CS team containing the prepared feedback information. This step will require configuring email sending capabilities (e.g., using an SMTP server or an email sending service).


**Reasoning**:
Import the necessary libraries for sending emails and define email credentials and SMTP server details.



In [14]:
import smtplib
from email.mime.text import MIMEText
import os

# Define your email credentials and SMTP server details
# Replace with your actual credentials and server details
sender_email = os.environ.get('SENDER_EMAIL') # Use environment variables for security
sender_password = os.environ.get('SENDER_PASSWORD') # Use environment variables for security
smtp_server = 'smtp.gmail.com' # Example for Gmail
smtp_port = 587 # Example for TLS
recipient_email = 'cs_team@example.com' # Replace with the recipient's email address

print("Email credentials and SMTP server details defined.")

Email credentials and SMTP server details defined.


**Reasoning**:
Create the email content, establish a connection to the SMTP server, log in, send the email, and close the connection, including error handling.



In [15]:
# Create an MIMEText object to represent the email body
email_body = MIMEText(email_body_content)
email_body['Subject'] = "Action Required: Customer Feedback Below Sentiment Threshold"
email_body['From'] = sender_email
email_body['To'] = recipient_email

try:
    # Establish a connection to the SMTP server and log in
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()  # Secure the connection
        server.login(sender_email, sender_password)

        # Send the email
        server.sendmail(sender_email, recipient_email, email_body.as_string())

    print("Email sent successfully!")

except Exception as e:
    print(f"An error occurred while sending the email: {e}")


An error occurred while sending the email: (334, b'UGFzc3dvcmQ6')


## Implement email sending retry 1

### Subtask:
Retry implementing the email sending process. Address the authentication error encountered in the previous attempt.


**Reasoning**:
Re-execute the code to establish a connection to the SMTP server, log in using the environment variables, and send the email with the prepared content to the recipient email address, including error handling.



In [16]:
# Create an MIMEText object to represent the email body
email_body = MIMEText(email_body_content)
email_body['Subject'] = "Action Required: Customer Feedback Below Sentiment Threshold"
email_body['From'] = sender_email
email_body['To'] = recipient_email

try:
    # Establish a connection to the SMTP server and log in
    with smtplib.SMTP(smtp_server, smtp_port) as server:
        server.starttls()  # Secure the connection
        server.login(sender_email, sender_password)

        # Send the email
        server.sendmail(sender_email, recipient_email, email_body.as_string())

    print("Email sent successfully!")

except Exception as e:
    print(f"An error occurred while sending the email: {e}")

An error occurred while sending the email: (334, b'UGFzc3dvcmQ6')


## Automate the process

### Subtask:
Consider how to automate the execution of this process (e.g., scheduling a script to run periodically) to continuously monitor new feedback and send triggers.


**Reasoning**:
Outline methods for automating the script execution and discuss the necessary considerations and limitations.



In [17]:
# Discuss potential methods for automating the execution of the script.
print("Potential methods for automating script execution:")
print("- Cron jobs (Linux/macOS): Schedule scripts to run at fixed times or intervals.")
print("- Task Scheduler (Windows): Similar to cron jobs, for scheduling tasks on Windows systems.")
print("- Cloud-based scheduling services (AWS CloudWatch Events, Google Cloud Scheduler, Azure Scheduler): Managed services for scheduling tasks in cloud environments.")
print("\nSteps for setting up automation (example using cron jobs):")
print("1. Ensure the Python script is executable and the necessary dependencies are installed in the environment where the cron job will run.")
print("2. Open the crontab editor (e.g., by running 'crontab -e' in the terminal).")
print("3. Add a line to specify the schedule and the command to execute the script. For example, to run daily at 9 AM:")
print("# 0 9 * * * /usr/bin/python /path/to/your/script.py")
print("   (This cron expression means: run at minute 0, hour 9, every day of the month, every month, every day of the week)")
print("4. Save and exit the crontab editor.")

print("\nPrerequisites and considerations for automation:")
print("- File access: Ensure the script has read access to the 'customer_feedback.xlsx' file.")
print("- Environment variables: Ensure the necessary environment variables for email sending (SENDER_EMAIL, SENDER_PASSWORD) are set in the environment where the script runs.")
print("- Authentication issue: The automation will only successfully send emails once the email sending authentication issue is resolved.")

print("\nAcknowledgement of current limitation:")
print("While the automation setup can be outlined, successful execution of the email sending process within the automated script depends on resolving the prior email sending authentication problem.")

Potential methods for automating script execution:
- Cron jobs (Linux/macOS): Schedule scripts to run at fixed times or intervals.
- Task Scheduler (Windows): Similar to cron jobs, for scheduling tasks on Windows systems.
- Cloud-based scheduling services (AWS CloudWatch Events, Google Cloud Scheduler, Azure Scheduler): Managed services for scheduling tasks in cloud environments.

Steps for setting up automation (example using cron jobs):
1. Ensure the Python script is executable and the necessary dependencies are installed in the environment where the cron job will run.
2. Open the crontab editor (e.g., by running 'crontab -e' in the terminal).
3. Add a line to specify the schedule and the command to execute the script. For example, to run daily at 9 AM:
# 0 9 * * * /usr/bin/python /path/to/your/script.py
   (This cron expression means: run at minute 0, hour 9, every day of the month, every month, every day of the week)
4. Save and exit the crontab editor.

Prerequisites and considera

## Summary:

### Data Analysis Key Findings

*   100% of the records in the filtered `negative_feedback_df` DataFrame have a sentiment score less than or equal to 0.4, with scores ranging from -0.02 to 0.40.
*   The process successfully formatted the 'Feedback', 'Customer Type', 'Timestamp', and 'sentiment_score' columns into a string suitable for an email body.
*   The email sending process failed due to an authentication error when attempting to log in to the SMTP server, preventing the email containing negative feedback from being sent to the CS team.

### Insights or Next Steps

*   The primary next step is to resolve the email sending authentication issue by verifying and correctly configuring the sender's email credentials and SMTP server settings, potentially using application-specific passwords if two-factor authentication is enabled.
*   Once the email sending is functional, the outlined automation methods (like cron jobs or cloud schedulers) can be implemented to ensure the script runs periodically and automatically notifies the CS team of negative feedback.
